In [22]:
import pandas as pd
import re
from geonamescache import GeonamesCache
from unidecode import unidecode
import numpy as np

In [4]:
df = pd.read_csv("outbreak_cities.csv", index_col=0)

In [5]:
df.head()

,Headline,City,Country
0,First Case of Zika in Miami Beach,Miami Beach,United States
1,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil
2,Zika Concerns are Spreading in Houston,Houston,United States
3,The CDC in Atlanta is Growing Worried,Atlanta,United States
4,Zika Infested Monkeys in Sao Paulo,Sao Paulo,Brazil


In [6]:
cities = list(df["City"])

## Getting Longitudes and Latitudes

The process for extracting longitudes and latitudes is as follows, the names of the cities will be used in accordance with the functions used to extract the country name, however, this time those functions will instead return the longitude and latitude.

In [23]:
all_countries = gc.get_countries()
accented_names = [city['name'] for city in gc.get_cities().values()
                  if city['name'] != unidecode(city['name'])]

In [24]:
def singular_country(possible_cities, countries):
    selected_city_dict = possible_cities[0]
    iterator = iter(selected_city_dict.keys())
    inner_dict = selected_city_dict[next(iterator)]
    lat = inner_dict["latitude"]
    long = inner_dict["longitude"]
    return (lat, long)

In [29]:
def lat_long(city_name, countries, accent_list):
    possible_cities = gc.get_cities_by_name(city_name)
    if len(possible_cities) == 1:
        return singular_country(possible_cities, countries)
    elif len(possible_cities) > 1:
        return multiple_countries(possible_cities, countries)
    else:
        return accented_city(city_name, countries, accent_list)
        
        

In [30]:
def multiple_countries(city_list, countries):
    best_sf = max(city_list,
              key=lambda x: list(x.values())[0]['population'])
    iterator = iter(best_sf.keys())
    inner_dict = best_sf[next(iterator)]
    lat = inner_dict["latitude"]
    long = inner_dict["longitude"]
    return (lat, long)

In [31]:
def accented_city(city_name, countries, accented_cities):
    for city in accented_cities:
        if unidecode(city) == city_name:
            
            return lat_long(city, countries, accented_cities)

        else:
            continue

In [11]:
gc = GeonamesCache()

In [12]:
dubai = gc.get_cities_by_name("Dubai")

In [32]:
lat_long("Dubai", all_countries, accented_names)

(25.07725, 55.30927)

In [34]:
lats_and_longs = [lat_long(city, all_countries, accented_names) for city in cities]

In [35]:
len(lats_and_longs)

650

In [36]:
list_of_nulls = len([item for item in lats_and_longs if item is None])

In [37]:
list_of_nulls

51

In [40]:
lats = []
longs = []

for tup in lats_and_longs:
    if tup is None:
        lats.append(None)
        longs.append(None)
        continue
    lats.append(tup[0])
    longs.append(tup[1])

In [41]:
len(lats)

650

In [42]:
df["Latitude"] = lats
df["Longitude"] = longs

In [43]:
df.to_csv("outbreaks_lat_longs.csv")